In [3]:
#import packages and create date

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import datetime
from datetime import *
import shutil
import difflib
from pulp import *
import openpyxl
import glob

today = date.today()
# today = today - datetime.timedelta(days=2)
today = today.strftime("%m.%d.20%y")
today

'01.28.2022'

In [8]:
## FanDuel Player Pool

contest = "PlayerPool_"
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
contest_data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/FD_Data/PlayerPool'

# fd = glob.glob('FanDuel-NBA*.csv')[0]
# os.rename(fd, "FDSalaries_"+ contest + str(today)+".csv")

# shutil.move(downloads + "/"+ "FDSalaries_"+ contest + str(today)+".csv",
#            contest_data_loc + "/"+ "FDSalaries_"+ contest + str(today)+".csv")

os.chdir(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/FD_Data/PlayerPool")
fanduel_file = pd.read_csv(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/FD_Data/PlayerPool/FDSalaries_"+ contest + str(today)+".csv")
fanduel_file.to_excel(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/FD_Data/PlayerPool/FDSalaries_"+ contest + str(today)+".xlsx", index=False)
os.remove("FDSalaries_"+ contest + str(today)+".csv")


In [9]:
## ETR Daily

# changes the working directory to your downloads folder
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/ETR_Data/FanDuel'
daily_file = "FD NBA Projections.csv"

shutil.move(downloads + "/"+ daily_file,
           data_loc + "/"+ daily_file)

os.chdir(data_loc)

df = pd.read_csv(daily_file)

df.to_excel("ETR_Daily_"+str(today)+".xlsx", index=False)
os.remove("FD NBA Projections.csv")

In [10]:
fd = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\FD_Data\PlayerPool\FDSalaries_PlayerPool_' + str(today) + ".xlsx")
fd.head()

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Unnamed: 14,Unnamed: 15,Roster Position
0,70933-9488,SF/PF,LeBron,LeBron James,James,52.908332,36,10700,LAL@CHA,LAL,CHA,O,Knee,NaN,NaN,NaN,SF/PF
1,70933-80808,SF/PF,Jayson,Jayson Tatum,Tatum,44.175556,45,9700,BOS@ATL,BOS,ATL,NaN,NaN,NaN,NaN,NaN,SF/PF
2,70933-84671,PG,Trae,Trae Young,Young,45.646513,43,9100,BOS@ATL,ATL,BOS,NaN,NaN,NaN,NaN,NaN,PG
3,70933-15755,PF/C,Anthony,Anthony Davis,Davis,46.627585,29,8900,LAL@CHA,LAL,CHA,O,Wrist,NaN,NaN,NaN,PF/C
4,70933-145485,PG,LaMelo,LaMelo Ball,Ball,42.488095,42,8600,LAL@CHA,CHA,LAL,NaN,NaN,NaN,NaN,NaN,PG


In [12]:
## Joins the FD and ETR Files to include FD Positions 
    ## ETR does not include G/F/UTIL

os.chdir(r"C:\Users\vchang\Documents\Fantasy Bball\DFS\PlayerPool Analysis")
etr = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\ETR_Data\FanDuel\ETR_Daily_' + str(today) + ".xlsx")
fd = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\FD_Data\PlayerPool\FDSalaries_PlayerPool_' + str(today) + ".xlsx")

## TO DO - check if merge on only DKNG Roster Position Works
df = pd.merge(etr, 
              fd[['Nickname','Position', 'Id']], 
              left_on='Player', 
              right_on='Nickname')

In [13]:
df.head()

,Player,Team,Opp,Minutes,FD Position,FD Salary,FD Points,FD Value,FDCeiling,FD Own,FD Slate,Nickname,Position,Id
0,Russell Westbrook,LAL,CHA,34.5,PG,"7,800",47.8,11.6,58.4,40.8,MAIN,Russell Westbrook,PG,70933-9642
1,LaMelo Ball,CHA,LAL,35.0,PG,"8,600",48.5,9.1,59.1,26.0,MAIN,LaMelo Ball,PG,70933-145485
2,Cody Martin,CHA,LAL,35.0,SG/PG,"4,300",27.0,4.8,35.8,27.4,MAIN,Cody Martin,SG/PG,70933-110351
3,Miles Bridges,CHA,LAL,36.0,SF,"7,700",39.7,3.9,48.6,15.6,MAIN,Miles Bridges,SF,70933-84678
4,Mason Plumlee,CHA,LAL,26.5,C,"4,000",24.3,3.3,32.2,4.4,MAIN,Mason Plumlee,C,70933-15590


In [14]:
# ## Unnests Roster Positions into each Row (explode in Python) ##


# ## Update Salary to INT 
df['FD Salary'] = df['FD Salary'].str.replace(",","").astype(float)

df['Roster Position'] = df['Position'].astype(str)
df['Roster Position'] = df['Position'].str.split('/')

df.head()

,Player,Team,Opp,Minutes,FD Position,FD Salary,FD Points,FD Value,FDCeiling,FD Own,FD Slate,Nickname,Position,Id,Roster Position
0,Russell Westbrook,LAL,CHA,34.5,PG,7800.0,47.8,11.6,58.4,40.8,MAIN,Russell Westbrook,PG,70933-9642,[PG]
1,LaMelo Ball,CHA,LAL,35.0,PG,8600.0,48.5,9.1,59.1,26.0,MAIN,LaMelo Ball,PG,70933-145485,[PG]
2,Cody Martin,CHA,LAL,35.0,SG/PG,4300.0,27.0,4.8,35.8,27.4,MAIN,Cody Martin,SG/PG,70933-110351,"[SG, PG]"
3,Miles Bridges,CHA,LAL,36.0,SF,7700.0,39.7,3.9,48.6,15.6,MAIN,Miles Bridges,SF,70933-84678,[SF]
4,Mason Plumlee,CHA,LAL,26.5,C,4000.0,24.3,3.3,32.2,4.4,MAIN,Mason Plumlee,C,70933-15590,[C]


In [15]:
df.to_excel("FD_final_PlayerPool_ETR_"+str(today)+".xlsx", index=False)
os.system("FD_final_PlayerPool_ETR_"+str(today)+".xlsx")

0

# Create the Constraint Problem

Goal: Maximize FD Points

- Total Players = 9
- TotalSalary <= 60000
- TotalPosition_PG
- TotalPosition_SG
- TotalPosition_SF
- TotalPosition_PF
- TotalPosition_C


In [16]:
prob = pulp.LpProblem('NBA_DFS_Daily', LpMaximize)

In [17]:
player_vars = [pulp.LpVariable(f'player_{row.Player}', cat='Binary') for row in df.itertuples()]

In [18]:
prob += pulp.lpSum(player_var for player_var in player_vars) == 9

In [19]:
# position constraints
def get_position_sum(player_vars, df, position):
    return pulp.lpSum([player_vars[i] * (position in df['FD Position'].iloc[i]) for i in range(len(df))])
    
# prob += get_position_sum(player_vars, df, 'PG') <= 2
prob += get_position_sum(player_vars, df, 'PG') >= 2
# prob += get_position_sum(player_vars, df, 'SG') <= 2
prob += get_position_sum(player_vars, df, 'SG') >= 2
# prob += get_position_sum(player_vars, df, 'G') <= 2
# prob += get_position_sum(player_vars, df, 'G') >= 2
# prob += get_position_sum(player_vars, df, 'SF') <= 2
prob += get_position_sum(player_vars, df, 'SF') >= 2
# prob += get_position_sum(player_vars, df, 'PF') <= 2
prob += get_position_sum(player_vars, df, 'PF') >= 2
# prob += get_position_sum(player_vars, df, 'F') <= 2
# prob += get_position_sum(player_vars, df, 'F') >= 2
prob += get_position_sum(player_vars, df, 'C') <= 1
prob += get_position_sum(player_vars, df, 'C') >= 1
# prob += get_position_sum(player_vars, df, 'UTIL') >= 2
# prob += get_position_sum(player_vars, df, 'UTIL') <= 2

#prob += get_position_sum(player_vars, df, 'PG') <= 3
#prob += get_position_sum(player_vars, df, 'PG') >= 1
#prob += get_position_sum(player_vars, df, 'SG') <= 3
#prob += get_position_sum(player_vars, df, 'SG') >= 1
#prob += get_position_sum(player_vars, df, 'G') == 1
#prob += get_position_sum(player_vars, df, 'SF') <= 3
#prob += get_position_sum(player_vars, df, 'SF') >= 1
#prob += get_position_sum(player_vars, df, 'PF') <= 3
#prob += get_position_sum(player_vars, df, 'PF') >= 1
#prob += get_position_sum(player_vars, df, 'F') == 1
#prob += get_position_sum(player_vars, df, 'C') <= 2
#prob += get_position_sum(player_vars, df, 'C') >= 1
#prob += get_position_sum(player_vars, df, 'UTIL') == 1

In [20]:
# total salary constraint

prob += pulp.lpSum(df['FD Salary'].iloc[i] * player_vars[i] for i in range(len(df))) <= 60000

In [21]:
# finally, specify the goal
prob += pulp.lpSum([df['FD Points'].iloc[i] * player_vars[i] for i in range(len(df))])

In [22]:
# solve and print the status
prob.solve()
print(LpStatus[prob.status])

Optimal


In [23]:
# for each of the player variables, 
total_salary_used = 0
mean_AvgPointsPerGame = 0
for i in range(len(df)):
    if player_vars[i].value() == 1:
        row = df.iloc[i]
        print(row['FD Position'], row.Player, row.Team, row['FD Salary'], row['FD Points'])
        total_salary_used += row['FD Salary']
        mean_AvgPointsPerGame += row['FD Points']
#mean_AvgPointsPerGame /= 9  # divide by total players in roster to get a mean
total_salary_used, mean_AvgPointsPerGame

PG Russell Westbrook LAL 7800.0 47.8
PG LaMelo Ball CHA 8600.0 48.5
SG/PG Cody Martin CHA 4300.0 27.0
C Mason Plumlee CHA 4000.0 24.3
SF Stanley Johnson LAL 3600.0 22.3
PG Trae Young ATL 9100.0 43.8
SG/PG Terry Rozier CHA 7700.0 37.1
SF/PF Carmelo Anthony LAL 5200.0 27.0
SF/PF Jayson Tatum BOS 9700.0 43.1


(60000.0, 320.9)

In [24]:
df.dtypes

Player              object
Team                object
Opp                 object
Minutes            float64
FD Position         object
FD Salary          float64
FD Points          float64
FD Value           float64
FDCeiling          float64
FD Own             float64
FD Slate            object
Nickname            object
Position            object
Id                  object
Roster Position     object
dtype: object